In [1]:
import tomotopy as tp
import pandas as pd

In [2]:
import nltk
from time import time
#nltk.download('punkt')
from nltk import word_tokenize 
from nltk.corpus import stopwords
#nltk.download('stopwords')
#nltk.download('words')
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import r2_score

## Caricamento del dataset

In [3]:
data_1 = pd.read_csv('C:/Users/micky/OneDrive/Desktop/Tesi DS/data/drugsComTest_raw.csv')
data_2 = pd.read_csv('C:/Users/micky/OneDrive/Desktop/Tesi DS/data/drugsComTrain_raw.csv')
data_1.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,163740,Mirtazapine,Depression,"""I&#039;ve tried a few antidepressants over th...",10,28-Feb-12,22
1,206473,Mesalamine,"Crohn's Disease, Maintenance","""My son has Crohn&#039;s disease and has done ...",8,17-May-09,17
2,159672,Bactrim,Urinary Tract Infection,"""Quick reduction of symptoms""",9,29-Sep-17,3
3,39293,Contrave,Weight Loss,"""Contrave combines drugs that were used for al...",9,5-Mar-17,35
4,97768,Cyclafem 1 / 35,Birth Control,"""I have been on this birth control for one cyc...",9,22-Oct-15,4


In [63]:
#data = data.append(data_2)
data = pd.concat([data_1,data_2])[['review','condition']]

In [64]:
print("Shape of the dataset: ", data.shape)

Shape of the dataset:  (215063, 2)


In [65]:
# sample 5%
data = data.sample(frac=.05, random_state=9, replace=False)
data.shape

(10753, 2)

In [66]:
data = data[pd.notnull(data['review'])]

In [67]:
count_df = data[['condition', 'review']].groupby('condition').aggregate({
    'review': 'count'
}).reset_index().sort_values('review', ascending=False)
count_df.head()

,condition,review
77,Birth Control,1852
135,Depression,625
324,Pain,418
53,Anxiety,403
29,Acne,346


In [68]:
target_conditions = count_df[count_df['review'] > 200]['condition'].values
target_conditions

array(['Birth Control', 'Depression', 'Pain', 'Anxiety', 'Acne',
       'Bipolar Disorde', 'Weight Loss', 'Insomnia', 'ADHD', 'Obesity'],
      dtype=object)

In [69]:
len(target_conditions)

10

In [70]:
data = data[data['condition'].apply(lambda x: True
                                    if x in target_conditions else False)]

In [71]:
data.shape #new shape

(4889, 2)

In [72]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data['condition'] = le.fit_transform(data['condition'])

In [24]:
data.condition.unique()

array([0, 8, 3, 5, 2, 9, 6, 7, 4, 1])

Conversione in minuscolo

In [25]:
data['review'] = data['review'].str.lower()

Rimozione numeri

In [26]:
data['review'] = data['review'].str.replace(r'[0-9]+', ' ', regex=True)

Rimozione link

In [27]:
data['review'] = data['review'].str.replace(r'http\S+', ' ', regex=True)

Rimozione punteggiatura

In [28]:
data['review'] = data['review'].str.replace(r'[^\w\s]', ' ', regex=True)

Rimozione caratteri speciali tranne le lettere

In [29]:
data['review'] = data['review'].str.replace(r'[^a-zA-Z0-9 ]', ' ', regex=True)

Rimozione lettere singole e doppie

In [30]:
data['review'] = data['review'].str.replace('\\b\\w{1,2}\\b', '', regex=True)

Rimozione spazi extra

In [31]:
data['review'] = data['review'].str.replace(' +', ' ', regex=True)

In [32]:
data['review']

68054      great medicine recently restarted this medici...
79399      had bad car wreck caused chronic back pain ha...
8053       did nothing for pain from fibromyalgia amp ha...
126235     started taking geodon last week has pulled ou...
6375       have tried what seems like every medicine for...
                                ...                        
129590     have been this medication for over three mont...
121083     was trying fall asleep for minutes then said ...
106235     had nexplanon for about year hate had trouble...
114773     today day phentermine only taking half the pi...
29462      increased quality life after trying several a...
Name: review, Length: 4889, dtype: object

Rimozione stopwords

In [33]:
stop = stopwords.words('english')
data['review'] = data['review'].apply(lambda x: [item for item in str(x).split() if item not in stop])

Stemming

In [34]:
stemmer = SnowballStemmer("english")
data['review'] = data['review'].apply(lambda x: [stemmer.stem(y) for y in x])
data['review']

68054     [great, medicin, recent, restart, medicin, yea...
79399     [bad, car, wreck, caus, chronic, back, pain, s...
8053      [noth, pain, fibromyalgia, amp, chronic, pain,...
126235    [start, take, geodon, last, week, pull, terrib...
6375      [tri, seem, like, everi, medicin, depress, exp...
                                ...                        
129590    [medic, three, month, moder, acn, got, rid, ac...
121083    [tri, fall, asleep, minut, said, ohh, crap, fo...
106235    [nexplanon, year, hate, troubl, sleep, much, s...
114773    [today, day, phentermin, take, half, pill, los...
29462     [increas, qualiti, life, tri, sever, antidepre...
Name: review, Length: 4889, dtype: object

Rimozione eventuali documenti vuoti

In [35]:
data = data[data['review'].map(len) != 0]

In [36]:
data.shape

(4889, 2)

### Train-test split

In [37]:
from sklearn.model_selection import train_test_split

In [38]:
data.head()

,review,condition
68054,"[great, medicin, recent, restart, medicin, yea...",0
79399,"[bad, car, wreck, caus, chronic, back, pain, s...",8
8053,"[noth, pain, fibromyalgia, amp, chronic, pain,...",8
126235,"[start, take, geodon, last, week, pull, terrib...",3
6375,"[tri, seem, like, everi, medicin, depress, exp...",5


In [44]:
#train-test split 80%-20%
X_train, X_test, y_train, y_test = train_test_split(data['review'],
                                                    data['condition'],
                                                    test_size=0.2,
                                                    random_state=1234)

In [46]:
slda = tp.SLDAModel(k=10, vars=['l'], seed=1234, rm_top=10, min_df=3, alpha=.1)

for document, rating in zip(X_train, y_train):
    slda.add_doc(document, [rating])

slda.burn_in = 10
slda.train(0)

print('Num docs:{}, Num Vocabs:{}, Total Words:{}'.format(
    len(slda.docs), len(slda.used_vocabs), slda.num_words))
print('Removed Top words: ', *slda.removed_top_words)

Num docs:3911, Num Vocabs:2839, Total Words:140111
Removed Top words:  take month day year week get start effect period feel


In [47]:
t0 = time()
slda.train(100)
print("done in %0.3fs." % (time() - t0))

done in 1.596s.


In [48]:
yhat = []
for document in X_test:
    doc = slda.make_doc(document)
    slda.infer(doc)
    yhat.append(slda.estimate(doc)[0])
rsq = r2_score(y_test, yhat)
print('R2 score sul test set per k={}: {}'.format(slda.k, rsq))

R2 score sul test set per k=10: 0.5264247495160956


In [73]:
for i in range(10):
    print(slda.get_topic_words(i, 30), "\n")

[('anxieti', 0.045218244194984436), ('depress', 0.04059953987598419), ('side', 0.01984388753771782), ('medic', 0.01938772015273571), ('help', 0.01704985834658146), ('medicin', 0.013058384880423546), ('life', 0.012773280031979084), ('sever', 0.012488174252212048), ('dose', 0.012260090559720993), ('drug', 0.012260090559720993), ('attack', 0.010777543298900127), ('increas', 0.010264354757964611), ('med', 0.009694144129753113), ('panic', 0.009237975813448429), ('also', 0.008952870965003967), ('symptom', 0.007983513176441193), ('suffer', 0.0076413871720433235), ('lexapro', 0.007299261167645454), ('less', 0.007014156319200993), ('disord', 0.007014156319200993), ('bipolar', 0.006957135628908873), ('ago', 0.006957135628908873), ('prescrib', 0.006557988468557596), ('better', 0.006557988468557596), ('amp', 0.006557988468557596), ('mood', 0.006272883154451847), ('work', 0.0061018201522529125), ('realli', 0.005987777840346098), ('daili', 0.005930757150053978), ('much', 0.005702672991901636)] 

[('